***
<center><h1 style="color:#28abb9;font-weight:bold;">Neighborhoods data cleaning<h1/><center/>

***

In [49]:
import lxml.html as lh
import pandas as pd
import numpy as np
print('imports completed')

imports completed


# Download wikipedia page

In [1]:
!wget wiki.txt https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('download complete')

--2021-01-01 11:41:15--  http://wiki.txt/
Resolving wiki.txt (wiki.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘wiki.txt’
--2021-01-01 11:41:15--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 91.198.174.192, 2620:0:862:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54812 (54K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  53.53K  --.-KB/s    in 0.08s   

2021-01-01 11:41:16 (674 KB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [54812/54812]

FINISHED --2021-01-01 11:41:16--
Total wall clock time: 0.7s
Downloaded: 1 files, 54K in 0.08s (674 KB/s)


# Extracting rows and columns

In [243]:
file=open('List_of_postal_codes_of_Canada:_M','r')
content=file.read()
table_data=lh.fromstring(content).xpath('//tr')
row_size=3
#extracting columns
columns=[]
i=0
for col in table_data[0]:
    i=i+1
    columns.append(col.text_content())
columns
#reformating columns array
j=0
for col in columns:
    columns[j]=col[:col.find('\n')]
    j=j+1
columns
rows=[]
#Extracting rows
for i in range(1,len(table_data)):
    row=table_data[i]
    if len(row)!=3:
        break
    rows.append(row.text_content())

for i,row in enumerate(rows):
    rows[i]=row[1:len(row)-1].split('\n\n')
rows[:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']]

# Dataframe creation

In [244]:
df=pd.DataFrame(data=rows,columns=columns,index=None)
df.head()
# removing null values
df.loc[df['Borough']=='Not assigned']=np.nan
df.dropna(axis=0,inplace=True)
df.reset_index()


,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [240]:
df.loc[df['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


# Editing not assigned neighbourhood


In [245]:
for index,row in df.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']=row['Borough']

df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [246]:
df.shape

(104, 3)